# Configuraciones iniciales (Grafica y Equipo)

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

#Si se esta trabajando en colab la variable debe quedar en falso
SERVER = True

# Importaciones

## Librerias

### Instaladas python

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider
import numpy as np
import random
%matplotlib inline
import matplotlib.pyplot as plt
from collections import namedtuple
if not SERVER:
    %tensorflow_version 2.x
import tensorflow as tf

### Propias y externas

In [3]:
if not SERVER:
    #Archivos py
    !git clone https://github.com/JefeLitman/SignLanguageTranslation_SLT.git
    #Organizacion archivos
    !mv /content/SignLanguageTranslation_SLT/utils /content/
    !mv /content/SignLanguageTranslation_SLT/models /content/
    #Elimino la carpeta sobrante
    !rm -rf /content/SignLanguageTranslation_SLT
#DatasetsLoaderUtils
!wget -q https://raw.githubusercontent.com/JefeLitman/VideoDataGenerator/master/DatasetsLoaderUtils.py -O DatasetsLoaderUtils.py
!mv DatasetsLoaderUtils.py utils/DatasetsLoaderUtils.py

In [5]:
from utils.DatasetsLoaderUtils import flow_from_tablePaths
from utils.preprocess_data import preprocessing_paths, preprocessing_sentences, table_paths_dataset
from utils.data_augmentation import frame_sampling
from utils.build_tf_data import build_datasets

## Datos

In [6]:
if not SERVER:
    !rm -rf /content/sample_data
    !wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Ph_Ys3O_vI93WeTkDqr5h6kTJm0CZ0Ub' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ph_Ys3O_vI93WeTkDqr5h6kTJm0CZ0Ub" -O boston201.zip && rm -rf /tmp/cookies.txt
    !unzip -q boston201.zip
    !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip -O word_vectors.zip
    !unzip -q word_vectors.zip
    # Monto el drive para poder tener donde guardar los datos
    from google.colab import drive
    drive.mount('/content/drive')

# Modelo SLT

## Parametros de la red

In [7]:
Args = namedtuple('Args', 'tx ty data loss pretrained prefetch_batch_buffer dataset verbose unitsSatt unitsEmbedding vocab_size rol videos_path rnnUnits denseUnits dropout recurrent_dropout inputShape optimizer type_frames batchSize epochs nIters lr momentum decay wDecay path2save name')

args = Args(tx=32, 
            ty=15,
            videos_path='../DataSets/boston201',#'/content/boston201',
            denseUnits=256,
            rnnUnits=256,
            unitsEmbedding=300,
            unitsSatt=169,
            vocab_size=150,
            loss ='SparseCategoricalCrossentropy_mask',
            rol = 'train',
            dropout=0.2,
            recurrent_dropout=0.2,
            inputShape=(32, 112, 112, 3),
            pretrained=None,#'vgg16',
            optimizer='adam',
            type_frames='jpg/',
            batchSize=1,
            epochs=10,
            nIters=10.0,
            lr=0.001,
            momentum=0.99,
            decay=0.1,
            wDecay=0.0005,
            path2save='path_to_save_model_results',
            name='SLT_Model',
            data= '../DataSets/boston201/data/', #'/content/boston201/data/',
            dataset='boston201',
            prefetch_batch_buffer = 5,
            verbose=0)

## Preparacion datos

In [17]:
paths_translation = [args.data+'translations.train',  
                         args.data+'translations.test']
paths_videos = [args.data+'pathsigns.train', 
                    args.data+'pathsigns.test']

# Processing sentences and paths
preprocessed_sentences, vocab = preprocessing_sentences(paths_translation, max_len=args.ty)
preprocessed_paths = preprocessing_paths(paths_videos, path2videos=args.videos_path, type_=args.type_frames)

#Creating table paths
table_paths=table_paths_dataset(preprocessed_paths, preprocessed_sentences)

Reading training file ...
training on train sentences to make the vocab ...
training text to index sequences ..
training index sequences to padded sequences ..
Computing same steps over dev and test sequences
Completed
Completed
Creating the table paths for flow_from_tablePaths ..


In [18]:
dataset = flow_from_tablePaths(table_paths, lambda x: x, args.inputShape[1:3])

In [79]:
train_gen = dataset.data_generator(1,args.inputShape[-1])
test_gen = dataset.data_generator(2,args.inputShape[-1])

In [80]:
table_paths.shape

(201, 3)

In [81]:
for video, label in test_gen:
    print(video.shape)
    break

(76, 112, 112, 3)


In [ ]:
def train_gen_sampling():
    for v, l in train_gen:
        for new_v in frame_sampling(v, args.inputShape[0]):
            yield new_v, l
def test_gen_sampling():
    for v, l in test_gen:
        for new_v in frame_sampling(v, args.inputShape[0]):
            yield new_v, l

In [84]:
len(frame_sampling(video, 32))

2

In [88]:
frame_sampling(video, 32)[1].shape

(32, 112, 112, 3)

## Creacion y estructuracion

## Entrenamiento